# University Related Coronavirus Sentiment Analysis
---

**Insert generic writeup here**

We are using the NLTK package in Python to do our natural language processing tasks in this project. Let's start with some basic setup

To begin with, there are many ways we can help the computer process our data. There are numerous ways which we can "clean" our data to make our final prediction more accurate. Let's get into some of these methods below

## Getting our Data
---

We have collected our data in csv format, so let's put it in a Pandas dataframe and pull out the Tweets and place them in a list

In [49]:
import pandas as pd
import nltk
# Extract content and date fields from csv
fields=['date', 'content']
d = pd.read_csv('combined_csv.csv', usecols=fields)

uni_related_tweets = d['content']

## Tokenizing the Data
---

First, we will do what is called "tokenizing." This process will take the Tweets as a whole, and split it into smaller subsections called tokens. These tokens make it much easier for machines to understand the context of the text when developing the model.

To demonstrate how tokenization works, lets tokenize some Tweets:

In [50]:
from nltk.tokenize import word_tokenize
print(uni_related_tweets[0])   # String
print()
print(word_tokenize(uni_related_tweets[0])) # The same string, tokenized

Marcus Yoder (HCB ‘92) has been appointed senior vice president of sales for @GameAccountNtwk, a leading business-to-business supplier of internet gambling software-as-a-service solutions. https://t.co/a8uu63502V https://t.co/WqS4p0PVN9

['Marcus', 'Yoder', '(', 'HCB', '‘', '92', ')', 'has', 'been', 'appointed', 'senior', 'vice', 'president', 'of', 'sales', 'for', '@', 'GameAccountNtwk', ',', 'a', 'leading', 'business-to-business', 'supplier', 'of', 'internet', 'gambling', 'software-as-a-service', 'solutions', '.', 'https', ':', '//t.co/a8uu63502V', 'https', ':', '//t.co/WqS4p0PVN9']


Let's go ahead and tokenize all of these tweets, and store them for use later:

In [51]:
for i, sentence in enumerate(uni_related_tweets):
    uni_related_tweets[i] = word_tokenize(uni_related_tweets[i])
uni_related_tweets

0       [Marcus, Yoder, (, HCB, ‘, 92, ), has, been, a...
1       [Congratulations, to, the, Haslam, College, of...
2       [#, FacultyFriday, Supply, chain, management, ...
3       [Due, to, COVID-19, ,, most, internships, have...
4       [Congratulations, to, MSCM, Tri-Con, grad, Nai...
                              ...                        
7038    [2, weeks, from, tonight, is, Torch, Night, .,...
7039    [This, is, great, news, Tom, ., Thanks, for, t...
7040    [Having, a, partner, who, is, the, love, of, y...
7041    [So, impressed, with, the, crowd, &, amp, ;, t...
7042    [More, good, news, about, UTK, from, US, News,...
Name: content, Length: 7043, dtype: object

As you can see above, the tokenization process took the original string and split it into smaller subsections known as tokens

## Normalizing the Data
---

Normalization, in terms of natural language processing, is the process of transforming a text into a canonical (standard) form.
For example, "gooood," and "gud" can be resolved to the normalized form "good." This can also apply for different tenses of the same word. For example, "ran," "runs," and "running" are all forms of "run."

<br/>

#### There are a few things at work here:

Stemming is the process of removing suffixes and prefixes from words. As an example, it reduces the inflection in words such as "troubled" and "troubles" to their root form "trouble."

Here are some stemming examples made using Porters Algorithm, one of the most common stemming algorithms:

<html>
<img src="Documents/StemmingExample.PNG" alt="drawing" width="275"/>
</html>

Lemmatization is similar to stemming, but rather than just cutting off the affixes, it will transform the word to it's root. As an example, it may transform the word "better" to "good."

Here are some examples of lemmatization using a dictionary mapping for the translations:

<html>
<img src="Documents/LemmatizationExample.PNG" alt="drawing" width="275"/>
</html>

<br/>
<br/>


This processing is essential for noisy social-media posts, as abbreviations and mispellings are very common!

We will be using lemmatization for our data, so let's download ```wordnet```, a lexical database, and ```averaged_perceptron_tagger```, which will help us in determining context

In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

Before using the lemmatizer, we must determine the context of each word within our tweets. To do this, we use what's called a tagging algorithm. Fortunately, NLTK provides a function for this.

Let's test it here:

In [33]:
from nltk.tag import pos_tag
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


Here are some common tags and their meaning:
- NNP: Noun, proper, singular
- NN: Noun, common, singular or mass
- IN: Preposition or conjunction, subordinating
- VBG: Verb, gerund or present participle
- VBN: Verb, past participle

Using the fact that tags starting with ``NN`` are typically nouns, and tags starting with ```VB``` are typically verbs, we can incorporate this into a function to lemmatize our data:

In [35]:
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

This function gets the tag of each token within the Tweet, and lemmatizes accordingly.
Let's test it here:

In [36]:
print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']
